In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import pickle
import warnings

: 

In [ ]:
df = pd.read_csv(r'C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\train.csv')
df

: 

In [8]:
df.shape

(20000, 17)

In [10]:
df.size

340000

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20000 non-null  int64  
 1   temperature         18999 non-null  float64
 2   irradiance          19013 non-null  float64
 3   humidity            20000 non-null  object 
 4   panel_age           18989 non-null  float64
 5   maintenance_count   18973 non-null  float64
 6   soiling_ratio       18990 non-null  float64
 7   voltage             19007 non-null  float64
 8   current             19023 non-null  float64
 9   module_temperature  19022 non-null  float64
 10  cloud_coverage      18990 non-null  float64
 11  wind_speed          20000 non-null  object 
 12  pressure            20000 non-null  object 
 13  string_id           20000 non-null  object 
 14  error_code          14088 non-null  object 
 15  installation_type   14972 non-null  object 
 16  effi

In [ ]:
df.describe().transpose() # this will select only the numeric rows

,count,mean,std,min,25%,50%,75%,max
id,20000.0,9999.500000,5773.647028,0.000000,4999.750000,9999.500000,14999.250000,19999.000000
temperature,18999.0,25.077241,12.513129,0.000000,16.853522,24.720345,32.848917,147.394168
irradiance,19013.0,501.273896,250.926590,-597.278646,332.227277,499.654730,668.416734,1537.810349
panel_age,18989.0,17.509758,10.097557,0.001264,8.777905,17.497731,26.340761,34.998379
maintenance_count,18973.0,4.012070,2.002268,0.000000,3.000000,4.000000,5.000000,15.000000
soiling_ratio,18990.0,0.698879,0.172244,0.400149,0.550654,0.697663,0.847838,0.999949
voltage,19007.0,16.242251,17.889031,0.000000,0.000000,12.350138,26.557322,494.279016
current,19023.0,1.713396,1.152953,0.000054,0.772311,1.558413,2.474744,7.315597
module_temperature,19022.0,29.923807,12.125405,0.000000,21.522124,29.857669,38.094943,65.000000
cloud_coverage,18990.0,51.378575,48.473664,0.000244,25.081618,49.704133,75.052824,1000.000000


In [21]:
df[df['efficiency'] == df['efficiency'].max()]

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
1616,1616,30.160975,1228.561162,26.450265215585077,3.988241,10.0,0.943199,17.583248,4.842217,29.477682,67.952173,8.36247200704732,1016.4608719691104,B2,NaN,fixed,0.987066


In [31]:
df['wind_speed'].value_counts() 


wind_speed
badval                42
error                 41
unknown               36
1.4004639058482033     1
4.676583637669918      1
                      ..
9.196479438626795      1
0.7985304351521505     1
7.767053830591635      1
3.0572137101487558     1
8.321732514520054      1
Name: count, Length: 19884, dtype: int64

In [32]:
df['pressure'].value_counts()

pressure
unknown               46
error                 45
badval                44
1012.8003752419361     1
1001.3287852158609     1
                      ..
1021.4074573669117     1
1008.7504815771696     1
1014.7401210245107     1
1006.4182950874194     1
1030.4971803983756     1
Name: count, Length: 19868, dtype: int64

In [38]:
df.loc[df['pressure'].isin(['badval','error', 'unknown'])]

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
40,40,47.973606,NaN,22.058748668559303,16.093618,4.0,0.428969,47.819612,3.264913,52.238395,80.213642,2.7854029076788285,unknown,B2,E01,tracking,0.515999
79,79,1.891977,422.663365,36.660789342772034,11.312273,2.0,0.483908,2.548296,0.479120,7.818356,14.493848,7.80719789753973,unknown,D4,NaN,fixed,0.484963
94,94,40.368922,78.166392,83.62935788403288,4.336180,6.0,0.841042,28.894706,0.070557,45.190447,76.587267,0.7230370811744813,unknown,D4,NaN,fixed,0.435985
187,187,12.942382,193.882742,14.825030046292975,23.979997,6.0,0.680431,0.000000,1.349382,18.503371,29.296724,2.5304860869622026,unknown,C3,E00,tracking,0.412498
228,228,26.726921,286.689162,99.433526840726,22.590687,4.0,0.735756,NaN,0.555327,30.436082,52.092448,11.216905103677677,badval,B2,E02,fixed,0.395336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19505,19505,37.167240,558.010651,6.431461610112487,24.904775,3.0,0.511923,10.350710,1.308435,38.203510,NaN,4.274765098164273,badval,D4,E00,NaN,0.441483
19543,19543,25.404976,481.071234,22.801095249618697,15.689368,0.0,0.416402,7.713185,2.775103,36.081079,81.419422,12.128268170331731,unknown,A1,E00,dual-axis,0.460354
19626,19626,39.784297,669.728364,27.975323166237022,32.703050,5.0,0.737207,70.823170,0.681198,41.566154,52.773546,8.494866395022155,error,B2,E02,NaN,0.542955
19760,19760,13.751447,483.938795,74.28739629809242,14.937022,4.0,0.588709,19.464720,1.905392,18.353161,23.103253,7.7107782580320094,badval,A1,E02,NaN,0.469591
